In [41]:
import math
import random

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def calculate_gradient(output_i, delta_j):
        return output_i * delta_j

def calculate_weight_delta(learning_rate, delta_j, bias=False, input_i=None):
    if bias:
        return learning_rate * delta_j
    else:
        return learning_rate * calculate_gradient(input_i, delta_j)

In [42]:
class Neuron():
    def __init__(self, name, weights, bias, output_neuron=False):
        self.name = name
        self.weights = [float(x) for x in weights]
        self.bias = float(bias)
        self.delta = None
        self.weight_deltas = weights
        self.bias_delta = None
        self.output_neuron = output_neuron
        self.previous_input = None

    def __str__(self):
        return f"Neuron '{self.name}' with input weights {self.weights}, bias {self.bias} and error {self.delta}."
    
    def output(self, input):
        self.previous_input = input
        if type(input) == int:
            input = [input]
        if len(input) != len(self.weights):
            raise ValueError("The number of inputs must be equal to the number of weights.")
        total = 0.0
        for i in range(len(input)):
            total += input[i] * self.weights[i]
        return sigmoid(total + self.bias)
    
    def calculate_deltas(self, input, learning_rate):
        for i in range(len(self.weights)):
            self.weight_deltas[i] = self.weights[i] - calculate_weight_delta(learning_rate, self.delta, input_i=input[i])
        self.bias_delta = self.bias - calculate_weight_delta(learning_rate, self.delta, bias=True)
    
    def update_error(self, input, desired=None, weights=None, deltas=None):
        output = self.output(input)
        if self.output_neuron:
            self.delta = output * (1 - output) * - (desired - output)
        else:
            self.delta = output * (1 - output) * sum([weight * delta for weight, delta in zip(weights, deltas)])
        
class NeuronLayer():
    def __init__(self, name, neurons):
        self.name = name
        self.neurons = neurons

    def __str__(self):
        return f"NeuronLayer '{self.name}' with {len(self.neurons)} neurons."

    def output(self, input):
        return [n.output(input) for n in self.neurons] if hasattr(self.neurons, '__iter__') else self.neurons.output(input)
        
class NeuronNetwork():
    def __init__(self, name, layers):
        self.name = name
        self.layers = layers

    def __str__(self):
        return f"NeuronNetwork '{self.name}' with {len(self.layers)} layers."

    def feed_forward(self, input):
        for layer in self.layers:
            # Feed forward
            input = layer.output(input)
        return input
    
    def back_propagate(self, desired, learning_rate):
        # Ik loop achterstevoren door het netwerk heen, dus ik begin bij de outputlayer
        for i in range(len(self.layers) - 1, -1, -1):
            for j in range(len(self.layers[i].neurons) - 1, -1, -1):
                cur_neuron = self.layers[i].neurons[j]
                # Eerst de output neurons, ik check voor de zekerheid of het een output neuron is en of het de laatste laag is
                if cur_neuron.output_neuron == True and i == len(self.layers) - 1:
                    output_neuron = cur_neuron
                    output_neuron.update_error(output_neuron.previous_input, desired)
                    output_neuron.calculate_deltas(output_neuron.previous_input, learning_rate)
                # Nu voor de hidden layers
                else:
                    weights = []
                    deltas = []
                    for neuron in self.layers[i + 1].neurons:
                        weights.append(neuron.weights[j])
                        deltas.append(neuron.delta)
                    cur_neuron.update_error(cur_neuron.previous_input, weights=weights, deltas=deltas)
                    cur_neuron.calculate_deltas(cur_neuron.previous_input, learning_rate)
        
    def update(self):
        for layer in self.layers:
            for neuron in layer.neurons:
                neuron.weights = neuron.weight_deltas
                neuron.bias = neuron.bias_delta

    def mean_squared_error(self, output, target):
        return (target - output) ** 2
        

    def train(self, inputs, targets, learning_rate, epochs):
        for i in range(epochs):
            total_loss = 0
            for j in range(len(inputs)):
                print("Input", inputs[j])
                print("Iteration", j + 1)
                output = self.feed_forward(inputs[j])[0]
                print("Output", output)
                self.back_propagate(targets[j], learning_rate)
                self.update()
                loss = self.mean_squared_error(output, targets[j])
                total_loss += loss
                print_network(self)
            average_loss = total_loss / len(inputs)
            print(f"Epoch {i + 1} - Loss: {average_loss}")

def print_network(network):
    print(f"Network '{network.name}'")
    for layer in network.layers:
        print(f"Layer '{layer.name}'")
        for neuron in layer.neurons:
            print(neuron)
        print()


In [43]:
o = Neuron("Output", [-0.5, 0.5], 1.5, output_neuron=True)

outputlayer = NeuronLayer("OutputLayer", [o])

AND_network = NeuronNetwork("AND", [outputlayer])

In [44]:
AND_train = [
    ([0, 0], 0),
    ([1, 0], 0),
    ([0, 1], 0),
    ([1, 1], 1)
]

inputs = [x[0] for x in AND_train]
targets = [x[1] for x in AND_train]

AND_network.train(inputs, targets, 1, 1000)

Input [0, 0]
Iteration 1
Output 0.8175744761936437
Network 'AND'
Layer 'OutputLayer'
Neuron 'Output' with input weights [-0.5, 0.5], bias 1.3780616675724573 and error 0.12193833242754278.

Input [1, 0]
Iteration 2
Output 0.7064203900276265
Network 'AND'
Layer 'OutputLayer'
Neuron 'Output' with input weights [-0.6465049644916311, 0.5], bias 1.2315567030808263 and error 0.14650496449163108.

Input [0, 1]
Iteration 3
Output 0.8496114313519779
Network 'AND'
Layer 'OutputLayer'
Neuron 'Output' with input weights [-0.6465049644916311, 0.39144357812605246], bias 1.123000281206879 and error 0.10855642187394754.

Input [1, 1]
Iteration 4
Output 0.7043166439773602
Network 'AND'
Layer 'OutputLayer'
Neuron 'Output' with input weights [-0.5849275132288176, 0.45302102938886596], bias 1.1845777324696924 and error -0.0615774512628135.

Epoch 1 - Loss: 0.4941815057207157
Input [0, 0]
Iteration 1
Output 0.7657698937286594
Network 'AND'
Layer 'OutputLayer'
Neuron 'Output' with input weights [-0.584927513

In [45]:
print([0, 0], AND_network.feed_forward([0, 0])[0] > 0.5)
print([1, 0], AND_network.feed_forward([1, 0])[0] > 0.5)
print([0, 1], AND_network.feed_forward([0, 1])[0] > 0.5)
print([1, 1], AND_network.feed_forward([1, 1])[0] > 0.5)


[0, 0] False
[1, 0] False
[0, 1] False
[1, 1] True


In [46]:
# Random weights
weights_f = [0.2, -0.4]
weights_g = [0.7, 0.1]
weights_o = [0.6, 0.9]



# Hidden layer
f = Neuron("F", weights_f, 0.0)
g = Neuron("G", weights_g, 0.0)

hiddenlayer = NeuronLayer("Hidden", [f, g])

# Output layer
o = Neuron("O", weights_o, 0.0, output_neuron=True)

outputlayer = NeuronLayer("Output", [o])

xor = NeuronNetwork("XOR", [hiddenlayer, outputlayer])

In [47]:
XOR_train = [
    ([1, 1], 0),
    ([0, 1], 1),
    ([1, 0], 1),
    ([0, 0], 0)
]

inputs = [x[0] for x in XOR_train]
targets = [x[1] for x in XOR_train]

xor.train(inputs, targets, 0.01, 1000)

Input [1, 1]
Iteration 1
Output 0.7091123021807664
Network 'XOR'
Layer 'Hidden'
Neuron 'F' with input weights [0.19978277443898382, -0.40021722556101624], bias -0.00021722556101620326 and error 0.021722556101620324.
Neuron 'G' with input weights [0.6997184027720387, 0.09971840277203871], bias -0.0002815972279612951 and error 0.02815972279612951.

Layer 'Output'
Neuron 'O' with input weights [0.5993415419863753, 0.8989907740176232], bias -0.0014627004475984509 and error 0.1462700447598451.

Input [0, 1]
Iteration 2
Output 0.6705830053775949
Network 'XOR'
Layer 'Hidden'
Neuron 'F' with input weights [0.19978277443898382, -0.4001123978774063], bias -0.00011239787740629302 and error -0.010482768360991023.
Neuron 'G' with input weights [0.6997184027720387, 0.09988161442031893], bias -0.00011838557968107469 and error -0.01632116482802204.

Layer 'Output'
Neuron 'O' with input weights [0.5996334957566843, 0.8993726922811025], bias -0.0007350135685508397 and error -0.07276868790476111.

Input 

In [48]:
print([0, 0], xor.feed_forward([0, 0])[0] > 0.5)
print([0, 1], xor.feed_forward([0, 1])[0] > 0.5)
print([1, 0], xor.feed_forward([1, 0])[0] > 0.5)
print([1, 1], xor.feed_forward([1, 1])[0] > 0.5)

[0, 0] False
[0, 1] False
[1, 0] True
[1, 1] True
